### Import some librairies

In [2]:
import pandas as pd

In [3]:
import os
import anndata
import pandas as pd
import anndata as ad
import torch
from cellot.utils.helpers import load_config
from cellot.utils.loaders import load
from torch.utils.data import DataLoader

### Load the model to predict


In [142]:
config = load_config('/Users/MacBook/stanford/cellot/results/test_new/model-cellot/config.yaml')
chkpt='/Users/MacBook/stanford/cellot/results/test_new/model-cellot/cache/model.pt'

(f, g), opts,loader=load(config, restore=chkpt)

source Unstim_Bcell
target Stim_1_Bcell
{'Unstim_Bcell': 'source', 'Stim_1_Bcell': 'target'}


### first method: same as in the training/evaluating

In [143]:
from cellot.data.cell import read_list
from cellot.data.cell import AnnDataDataset
unstim_data_path = "/Users/MacBook/stanford/cellot/datasets/atest_data/unstim_Bcell_to_predict.h5ad"
unstim_anndata_to_predict = ad.read(unstim_data_path)
features = read_list(config.data.features)
unstim_anndata_to_predict = unstim_anndata_to_predict[:, features].copy()

dataset_args = {}
dataset2 = AnnDataDataset(unstim_anndata_to_predict.copy(), **dataset_args)

g.eval()
loader = DataLoader(dataset2, batch_size=len(dataset2), shuffle=False)
inputs = next(iter(loader))
outputs = g.transport(inputs.requires_grad_(True)).detach().numpy()
predicted_df2=anndata.AnnData(
            outputs,
            obs=dataset2.adata.obs.copy(),
            var=dataset2.adata.var.copy(),
        ).to_df()

### 2nd method: predict using torch (gives the same result)

In [ ]:
def predict_from_unstim_data(result_path, unstim_data_path, output_path):
    config_path = os.path.join(result_path, "config.yaml")
    chkpt = os.path.join(result_path, "cache/model.pt")

    # load the config and then the model (f,g)
    config = load_config(config_path)
    (f, g), opts, loader = load(config, restore=chkpt)
    g.eval()
    # load the data to predict and filter with the interzsting markers
    unstim_anndata_to_predict = ad.read(unstim_data_path)
    features = read_list(config.data.features)
    unstim_anndata_to_predict = unstim_anndata_to_predict[:, features].copy()

    # predict the data
    source_tensor = torch.tensor(unstim_anndata_to_predict.X, dtype=torch.float32).requires_grad_(True)
    transported_tensor = g.transport(source_tensor)
    transported_tensor = transported_tensor.detach()
    prediction = pd.DataFrame(transported_tensor.numpy())

    return prediction


result_path = "/Users/MacBook/stanford/cellot/results/test_new/model-cellot"
unstim_data_path = (
    "/Users/MacBook/stanford/cellot/datasets/atest_data/unstim_Bcell_to_predict.h5ad"
)
output_path = "/Users/MacBook/stanford/cellot/datasets/atest_data/prediction2.csv"
pred2=predict_from_unstim_data(result_path, unstim_data_path, output_path)

source Unstim_Bcell
target Stim_1_Bcell
{'Unstim_Bcell': 'source', 'Stim_1_Bcell': 'target'}
